# <center> Лабораторна робота №12. Прогнозування затримок вильоту літаків з використанням різних алгоритмів бустінгу

In [3]:
# !pip install xgboost

   ---------------------------------------- 99.7/99.7 MB 6.2 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [2]:
train = pd.read_csv("data/flight_delays_train.csv")
test = pd.read_csv("data/flight_delays_test.csv")

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Отже, потрібно за часом вильоту літака, коду авіакомпанії-перевізника, місця вильоту та прильоту та відстанню між аеропортами вильоту та прильоту передбачити затримку вильоту більше 15 хвилин. Як найпростіший бенчмарк візьмемо логістичну регресію та дві ознаки, які найлегше взяти: `DepTime` та `Distance`. У такої моделі результат – 0.68202.

In [5]:
X_train, y_train = (
    train[["Distance", "DepTime"]].values,
    train["dep_delayed_15min"].map({"Y": 1, "N": 0}).values,
)
X_test = test[["Distance", "DepTime"]].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train, y_train, test_size=0.3, random_state=17
)

In [6]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

np.float64(0.6795697123357751)

In [7]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, name="dep_delayed_15min").to_csv(
    "logit_2feat.csv", index_label="id", header=True
)

Побудувати покращену модель з використанням наступних підказок:
- ознаки `Distance` та `DepTime` брати без змін;
- створена ознака "маршрут" з вхідних ознак `Origin` та `Dest`;
- до ознак `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` і "маршрут" застосувати OHE-перетворення (`LabelBinarizer`);
- видділити відкладену вибірку;
- навчати логістичну регресію і градієнтний бустінг (xgboost або sklearn.ensemble.GradientBoostingRegressor), гіперпараметри бустінгу налаштувати за результатами крос-валідації, спочатку ті, що відповідають за складність моделі, потім число дерев зафіксувати рівним 500 і налаштовувати крок градієнтного спуску;
- за допомогою `cross_val_predict` сформувати прогнози обох моделей на крос-валідації (саме передбачення ймовірності), налаштувати лінійну суміш відповідей логістичної регресії і градієнтного бустінгу вигляду $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, де $p_{logit}$ – прогноз логістичною регресією ймовірності класу 1, $p_{xgb}$ – аналогічно. Вага $w_1$ підбирається вручну. 
- як відповідь на тестовій вибірці брати аналогічну комбінацію відповідей двух моделей, але вже навчених на всій навчальній вибірці.

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier

#### Створюємо ознаку "маршрут"

In [17]:
train['Route'] = train['Origin'] + "_" + train['Dest']
test['Route'] = test['Origin'] + "_" + test['Dest']

#### Виконуємо OHE-перетворення для категоріальних ознак

In [18]:
ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Route']
X_train_cat = ohe.fit_transform(train[categorical_features])
X_test_cat = ohe.transform(test[categorical_features])

#### Об'єднуємо числові та категоріальні ознаки

In [19]:
from scipy.sparse import hstack

X_train_full = hstack([X_train, X_train_cat])
X_test_full = hstack([X_test, X_test_cat])

#### Розділяємо дані на навчальну та валідаційну вибірки

In [20]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train_full, y_train, test_size=0.3, random_state=17
)

#### Навчаємо логістичну регресію

In [23]:
scaler = StandardScaler(with_mean=False)
X_train_part_scaled = scaler.fit_transform(X_train_part)
X_valid_scaled = scaler.transform(X_valid)

logit = LogisticRegression(random_state=17, max_iter=1000)
logit.fit(X_train_part_scaled, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid_scaled)[:, 1]

#### Навчаємо градієнтний бустинг

In [24]:
xgb = GradientBoostingClassifier(n_estimators=500, random_state=17)
xgb.fit(X_train_part, y_train_part)
xgb_valid_pred = xgb.predict_proba(X_valid)[:, 1]

#### Комбінуємо прогнози


In [25]:
w1 = 0.5 
combined_valid_pred = w1 * logit_valid_pred + (1 - w1) * xgb_valid_pred

#### Оцінюємо комбіновану модель


In [26]:
roc_auc = roc_auc_score(y_valid, combined_valid_pred)
print(f"ROC AUC комбінованої моделі: {roc_auc}")


ROC AUC комбінованої моделі: 0.7079796043422288


#### Навчаємо на повних навчальних даних

In [28]:

X_train_full_scaled = scaler.fit_transform(X_train_full)
logit = LogisticRegression(random_state=17, max_iter=2000)
logit.fit(X_train_full_scaled, y_train)
xgb.fit(X_train_full, y_train)

GradientBoostingClassifier(n_estimators=500, random_state=17)

#### Прогнозуємо на тестових даних

In [29]:
logit_test_pred = logit.predict_proba(X_test_full)[:, 1]
xgb_test_pred = xgb.predict_proba(X_test_full)[:, 1]

#### Комбінуємо прогнози тестових даних

In [30]:
combined_test_pred = w1 * logit_test_pred + (1 - w1) * xgb_test_pred

#### Зберігаємо прогнози

In [31]:
pd.Series(combined_test_pred, name="dep_delayed_15min").to_csv(
    "combined_model_predictions.csv", index_label="id", header=True
)